# Projeto 2 - Ciência dos Dados

Nome: Ellen Beatriz Shen

Nome: Gabriela Moreno Boriero

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re 

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [2]:
tudo = pd.read_excel("base_acai.xlsx")
serie_tudo = pd.Series(tudo.Treinamento)
tabela_tudo_absoluta = serie_tudo.value_counts()

In [3]:
def cleanup(text):
    #Função de limpeza muito simples que troca alguns sinais básicos por espaços
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [4]:
relevante_acai = cleanup((open("gosta.txt", "r", encoding="utf8").read()).lower())
irrelevante_acai = cleanup((open("nao_gosta.txt", "r", encoding="utf8").read()).lower())

In [6]:
tabela_relevante = (pd.Series(relevante_acai.split())).value_counts()
tabela_irrelevante = (pd.Series(irrelevante_acai.split())).value_counts()

In [8]:
relevante_acai_relativa = (pd.Series(relevante_acai.split())).value_counts(True)
irrelevante_acai_relativa = (pd.Series(irrelevante_acai.split())).value_counts(True)

relevante_acai_absoluta = (pd.Series(relevante_acai.split())).value_counts(False)
irrelevante_acai_absoluta = (pd.Series(irrelevante_acai.split())).value_counts(False)

In [9]:
probabilidade_relevante = tabela_relevante.sum()/tabela_tudo_absoluta.sum()
probabilidade_irrelevante = tabela_irrelevante.sum()/tabela_tudo_absoluta.sum()

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [10]:
def laplace(palavra, tabela1, tabela2):
    if palavra in tabela1:
        x = tabela1[palavra]
    else:
        x = 0
        
    if palavra in tabela2:
        y = tabela2[palavra]
    else:
        y = 0 
    
    prob_relevante = (x + 1)/ (len(tabela1) + len(set(tabela1+tabela2)))
    prob_irrelevante = (y + 1)/ (len(tabela2) + len(set(tabela1+tabela2)))
    return [prob_relevante,prob_irrelevante]    

In [17]:
base_teste = pd.read_excel("base_acai.xlsx",sheets="Teste")
series_tweets = pd.Series(list(base_teste.Treinamento))

tweets_limpos = []
for tweet in series_tweets:
    tweets_limpos.append(cleanup(tweet.lower()))

    
classificacao = {}
for frase in tweets_limpos:
    palavras_da_frase = list(frase.split())
    relevante = 1
    irrelevante = 1
    
    for palavra in palavras_da_frase:

        if palavra in relevante_acai_relativa and palavra in irrelevante_acai_relativa:
            relevante *= relevante_acai_relativa[palavra]
            irrelevante *= irrelevante_acai_relativa[palavra]
            
        elif palavra in relevante_acai_relativa and palavra not in irrelevante_acai_relativa:
            relevante *= relevante_acai_relativa[palavra]
            irrelevante *= laplace(palavra,relevante_acai_relativa,irrelevante_acai_relativa)[1]
        
        elif palavra in irrelevante_acai_relativa and palavra not in relevante_acai_relativa:
            relevante *= laplace(palavra,relevante_acai_relativa,irrelevante_acai_relativa)[0]
            irrelevante *= irrelevante_acai_relativa[palavra]
            
        else:
            relevante *= laplace(palavra,relevante_acai_relativa,irrelevante_acai_relativa)[0]
            irrelevante *= laplace(palavra,relevante_acai_relativa,irrelevante_acai_relativa)[1]
            
    prob_relevante = probabilidade_relevante*relevante
    prob_irrelevante = probabilidade_irrelevante*irrelevante
    
    if prob_relevante > prob_irrelevante:
        classificacao[frase] = 0
    else:
        classificacao[frase] = 1
       
classificacao
# teste_classificado = pd.Series(classificacao)
# teste_classificado.to_excel("Classificados.xlsx")

{'  era só um açaí 🤤': 0,
 '  ontem minha sogra compro açaí pra nós 🥰😍': 0,
 '[05/09/2019 09 54] cauã  vai fazer oque esse final de semana   \n [05/09/2019 09 55] cauã  nada né  se quiser a gente toma um açaí na praça pago p vc \n [05/09/2019 09 55] eduarda leal  já disse que você é um ótimo amigo hoje gato ': 0,
 '[5/9 09 06] guilherme ❤  hj vc quer\n[5/9 09 06] guilherme ❤  acai \n[5/9 09 06] guilherme ❤  sanduíche\n[5/9 09 06] guilherme ❤  ou cinema\n\neu amo  sério ': 1,
 '@_alves_crf vai pagar o açaí hj': 0,
 '@_bispob sim mana kkkkkkkkkkkk  depois acabou com meu açaí': 0,
 '@_vinicin_10 kkkkkkkkkkkkkk gostou né  só um açaí de 700ml agora🤤': 0,
 '@10maisogole açaí eu amo rs': 0,
 '@2cjoao açaí joão quero açaí': 0,
 '@ahprincipiokim quero um açaí também': 0,
 '@alemarinns oh dózinha  bora toma um açaí bb': 0,
 '@antibiotico_ ai nego  leite em pó é uó no açaí sai daqui\npaçoca  leite condensado e morango é everything to me': 1,
 '@batdeh nao acontecera por que nos vamos se divertir 

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**